In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llama-3.1/transformers/8b-instruct/2/model.safetensors.index.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00003-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/LICENSE
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00001-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/README.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/USE_POLICY.md
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/tokenizer_config.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00004-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/special_tokens_map.json
/kaggle/input/llama-3.1/transformers/8b-instruct/2/.gitattributes
/kaggle/input/llama-3.1/transformers/8b-instruct/2/model-00002-of-00004.safetensors
/kaggle/input/llama-3.1/transformers/8b-instruct/2/gener

In [2]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 23.8 MB/s eta 0:00:00:00:0100:01


In [3]:
import json
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, DataCollatorForLanguageModeling, Trainer
from peft import LoraConfig, get_peft_model
from datasets import Dataset
import torch

In [4]:
df = pd.read_csv("/kaggle/input/multi-lingual-sentiment-analysis/train.csv")

# Format dataset into instruction-response structure
formatted_data = []
for _, row in df.iterrows():
    language = row['language']
    sentence = row['sentence']
    label = row['label']
    
    formatted_data.append({"language": language, "sentence":sentence, "label": label})

# Save as JSON for training
with open("finetune_data.json", "w", encoding="utf-8") as f:
    json.dump(formatted_data, f, ensure_ascii=False, indent=4)

In [5]:
from datasets import load_dataset

lmap = {
    "as": "Assamese",
    "bd": "Bodo",
    "bn": "Bengali",
    "gu": "Gujarati",
    "hi": "Hindi",
    "kn": "Kannada",
    "ml": "Malayalam",
    "mr": "Marathi",
    "or": "Odia",
    "pa": "Punjabi",
    "ta": "Tamil",
    "te": "Telugu",
    "ur": "Urdu"
}

dataset = load_dataset("json", data_files={"train": "finetune_data.json"})

def format_example(example):
    sentiment = example['label']
    prompt = f"Analyze the sentiment of the following text written in {lmap[example['language']]}:\n\n{example['sentence']}\n\nSentiment:"
    return {"prompt": prompt, "completion": f" {sentiment}"}

formatted_dataset = dataset.map(format_example,remove_columns=dataset["train"].column_names)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [6]:
formatted_dataset['train'][0]

{'prompt': 'Analyze the sentiment of the following text written in Bengali:\n\nকর্মীদের ভাল আচরণ এবং খাবারের পাশাপাশি পানীয় (ককটেল এবং মকটেল) সহ একটি অনন্য জায়গা খুবই ভাল। প্রায়ই একটি সরাসরি সঙ্গীত পরিবেশনের সাথে এমন পরিবেশ তৈরী করে যে একজন দিন এবং সন্ধ্যা উভয় সময়েই জায়গাটি উপভোগ করতে পারে।\n\nSentiment:',
 'completion': ' Positive'}

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_path = "/kaggle/input/llama-3.1/transformers/8b-instruct/2"  # replace with the correct path or HF repo id

bnb_config = BitsAndBytesConfig(
    load_in_8bit=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map="auto",
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(model_path)

tokenizer.pad_token = tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
from peft import LoraConfig, get_peft_model

# Configure LoRA settings (adjust hyperparameters as needed)
lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=["q_proj", "v_proj"],  # adjust depending on the model architecture
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)

In [9]:
def tokenize_function(example):
    text = example["prompt"] + example["completion"] + tokenizer.eos_token
    tokenized = tokenizer(text, truncation=True, max_length=512, padding="max_length")

    # Labels should be the same as input_ids
    tokenized["labels"] = tokenized["input_ids"].copy()

    return tokenized


tokenized_dataset = formatted_dataset.map(tokenize_function, remove_columns=formatted_dataset["train"].column_names)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [10]:
print(tokenized_dataset["train"][99])

{'input_ids': [128000, 2127, 56956, 279, 27065, 315, 279, 2768, 1495, 5439, 304, 25578, 689, 1473, 156, 105, 237, 156, 105, 117, 156, 105, 122, 156, 105, 108, 29082, 105, 237, 156, 105, 243, 29082, 105, 229, 156, 105, 101, 156, 100465, 156, 105, 123, 156, 105, 110, 156, 255, 235, 156, 105, 253, 29082, 105, 227, 156, 105, 110, 156, 105, 253, 156, 255, 235, 156, 105, 108, 156, 105, 122, 29082, 100465, 156, 105, 122, 156, 34643, 156, 255, 235, 29082, 105, 231, 156, 105, 104, 156, 105, 108, 156, 255, 235, 29082, 105, 227, 156, 105, 249, 156, 105, 123, 765, 29082, 105, 245, 156, 255, 233, 156, 105, 253, 156, 105, 123, 156, 105, 237, 29082, 34643, 156, 255, 235, 156, 105, 103, 156, 105, 123, 156, 105, 243, 156, 105, 108, 156, 105, 108, 156, 255, 229, 29082, 105, 228, 156, 105, 106, 156, 255, 229, 29082, 105, 237, 156, 105, 243, 29082, 105, 103, 156, 105, 122, 156, 105, 108, 156, 255, 235, 156, 105, 253, 156, 255, 222, 29082, 34643, 156, 255, 229, 156, 105, 253, 156, 255, 235, 29082, 105, 108

In [11]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./llama_sentiment_finetuned",
    per_device_train_batch_size=2,  # adjust based on your GPU memory
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_steps=1,
    save_strategy="steps",
    save_steps=50,
    weight_decay=1e-3,
    fp16=True,  # if supported
    report_to="none"  # disable logging services if not needed
)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    processing_class=tokenizer
)

In [13]:
trainer.train()

Step,Training Loss
1,37.723400
2,34.573100
3,35.086900
4,32.744800
5,36.980100
6,27.741600
7,25.573900
8,19.532800
9,17.320300
10,12.720000


TrainOutput(global_step=186, training_loss=4.156728979079954, metrics={'train_runtime': 3159.2232, 'train_samples_per_second': 0.95, 'train_steps_per_second': 0.059, 'total_flos': 6.83671333896192e+16, 'train_loss': 4.156728979079954, 'epoch': 2.96})

In [14]:
model.save_pretrained("./llama_sentiment_finetuned")
tokenizer.save_pretrained("./llama_sentiment_finetuned")

('./llama_sentiment_finetuned/tokenizer_config.json',
 './llama_sentiment_finetuned/special_tokens_map.json',
 './llama_sentiment_finetuned/tokenizer.json')

In [15]:
test_data = pd.read_csv("/kaggle/input/multi-lingual-sentiment-analysis/test.csv")

In [16]:
def prompt(example):
    prompt = f"Analyze the sentiment of the following text written in {lmap[example['language']]}:\n\n{example['sentence']}\n\nSentiment:"
    return prompt

In [17]:
import re

def extract_sentiment(text):
    # Regex pattern to capture "Positive" or "Negative" (case-insensitive)
    pattern = r"Sentiment:\s*(Positive|Negative)"
    match = re.search(pattern, text, re.IGNORECASE)
    if match:
        return match.group(1)
    else:
        return "Positive"

In [18]:
input_text = prompt(test_data.iloc[98])
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=20)
result = tokenizer.decode(outputs[0], skip_special_tokens=True)
sentiment = extract_sentiment(result)
print(sentiment)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Positive


In [19]:
label = []
ID = []

for _, i in test_data.iterrows():
    input_text = prompt(i)
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=20)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    label.append(extract_sentiment(result))
    ID.append(i['ID'])
    print(result)

output_data = pd.DataFrame({'ID': ID, 'label': label})

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Analyze the sentiment of the following text written in Punjabi:

1120 mAh, ਓਵਰਚਾਰਜਿੰਗ ਦੀ ਸੁਰੱਖਿਆ

Sentiment: Positive


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Analyze the sentiment of the following text written in Gujarati:

તે સઘન મોઇશ્ચરાઇઝિંગ પ્રદાન કરે છે અને સરસ સ્વર્ગીય ફળની સુગંધ આપે છે

Sentiment: Positive


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Analyze the sentiment of the following text written in Odia:

ଏହା ଭଲ କ୍ୱାଲିଟିର ଗଦି ସହିତ ଭଲ ପ୍ୟାଡେଡ୍ ହୋଇଛି |

Sentiment: Positive


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Analyze the sentiment of the following text written in Malayalam:

അടുത്തിടെ ഞാൻ "Unfold" എന്നതിൽ നിന്ന് ഒരു ടെംപ്ലേറ്റ് വാങ്ങി, പക്ഷേ അത് ആക്‌സസ് ചെയ്യാനോ എന്റെ എഡിറ്റിംഗ് പേജ് കാണാനോ കഴിഞ്ഞില്ല. ഇത് തികച്ചും അലോസരപ്പെടുത്തുന്നതാണ്, ബാങ്ക് അക്കൗണ്ടിൽ നിന്ന് പണം കിഴിവ് ചെയ്തു, ടെംപ്ലേറ്റ് ഡോൺലോഡ് ചെയ്തു എന്ന് കാണിക്കുന്നു ഡൗൺലോഡ് ചെയ്‌തു, പക്ഷേ എഡിറ്റിംഗ് മോഡിൽ അത് പ്രതിഫലിക്കുന്നില്ല.

Sentiment: Negative


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Analyze the sentiment of the following text written in Malayalam:

"സംസ്ഥാനത്ത് നല്ല ബന്ധമാണുള്ളത്. നിങ്ങൾക്ക് സംസ്ഥാനത്തെ മിക്കവാറും എല്ലാ വലിയ നഗരങ്ങളിലേക്കും ചില അന്തർസംസ്ഥാന ലക്ഷ്യസ്ഥാനങ്ങളിലേക്കും അവരുടെ സേവനങ്ങൾ ഉപയോഗിച്ച് പോകാം."

Sentiment: Positive
Analyze the sentiment of the following text written in Bodo:

गोबां बोसोर सिगांनिफ्रायनो आं बेखौ मोजां मोनबोदों आरो बेयो गावनि मासिखौ दाबो लाखिना दं। बेयो एलकहल गैयि आरो देहानि मोनामनायखौ हमथाना लाखियो।

Sentiment: Positive


In [20]:
output_data

,ID,label
0,1,Positive
1,2,Positive
2,3,Positive
3,4,Positive
4,5,Negative
...,...,...
95,96,Positive
96,97,Positive
97,98,Negative
98,99,Positive


In [21]:
output_data.to_csv('output.csv',index=False)
print("done")

done
